In [1]:
from qiskit_nature.units import DistanceUnit
from qiskit_nature.second_q.drivers import PySCFDriver
from qiskit_nature.second_q.mappers import JordanWignerMapper, InterleavedQubitMapper
from qiskit_nature.second_q.circuit.library import HartreeFock, UCCSD
from qiskit_nature.second_q.operators import FermionicOp
from qiskit_algorithms import VQE
from qiskit_algorithms.optimizers import SLSQP, COBYLA, P_BFGS,ADAM,AQGD,NFT
from qiskit.primitives import Estimator

from produce_xacc_ham import geom_scf2qis
from moles import *

This is a modified operator_pools.py by Muqing Zheng
This is a modified vqe_methods.py by Muqing Zheng
This is a modified pyscf_helper.py by Muqing Zheng


In [2]:
# mole_obj = LiH()

# geom = "Li 0 0 0; H 0 0 2.5"

# a = 1.8
# geom = f"H 0 0 0; H {a} 0 0; H {a*2} 0 0; H {a*3} 0 0; H {a*4} 0 0; H {a*5} 0 0"


a = 1.5
geom = f"H 0 0 0; H {a} 0 0; H {a*2} 0 0; H {a*3} 0 0"

print("Geometry: ", geom)
# unit = DistanceUnit.BOHR
unit = DistanceUnit.ANGSTROM
##
driver = PySCFDriver(
    atom=geom,
    basis="sto3g",
    charge=0,
    spin=0,
    unit=unit,
)
es_problem = driver.run()
mapper = JordanWignerMapper()



qiskit_ham = es_problem.hamiltonian
## Add Nuclear Repulsion to Hamiltonian
from qiskit_nature.second_q.operators import PolynomialTensor
qiskit_ham.electronic_integrals.alpha += PolynomialTensor({
    "": qiskit_ham.nuclear_repulsion_energy
})
qiskit_ham.nuclear_repulsion_energy = None
##
qiskit_ferm_ham = qiskit_ham.second_q_op()
qiskit_ham_mat = mapper.map(qiskit_ferm_ham).to_matrix(sparse=True)
true_lowest_ev = scipy.sparse.linalg.eigsh(qiskit_ham_mat,1,which='SA')[0][0].real
print("True Lowest Eigenvalue: ", true_lowest_ev)

Geometry:  H 0 0 0; H 1.5 0 0; H 3.0 0 0; H 4.5 0 0
True Lowest Eigenvalue:  -1.9961503255188096


In [7]:
ansatz = UCCSD(
    es_problem.num_spatial_orbitals,
    es_problem.num_particles,
    mapper,
    initial_state=HartreeFock(
        es_problem.num_spatial_orbitals,
        es_problem.num_particles,
        mapper,
    ),
)
##
vqe_solver = VQE(Estimator(), ansatz,  P_BFGS(maxfun=1000, iprint=10) ) ## COBYLA(maxiter=2000, disp=True) # P_BFGS(maxfun=100, iprint=10) => 3m 21s, error 0.06611099896936112, NFT(maxiter=10, disp=True) => 50s,0.38112909977208664
vqe_solver.initial_point = [0.0] * ansatz.num_parameters
##
from qiskit_nature.second_q.algorithms import GroundStateEigensolver
calc = GroundStateEigensolver(mapper, vqe_solver)
res = calc.solve(es_problem)
print(res)

/var/folders/74/7bbhmvxd7mb2xrnkplmh90xc0000gn/T/ipykernel_82966/1193233684.py:12: DeprecationWarning: The class ``qiskit.primitives.estimator.Estimator`` is deprecated as of qiskit 1.2. It will be removed no earlier than 3 months after the release date. All implementations of the `BaseEstimatorV1` interface have been deprecated in favor of their V2 counterparts. The V2 alternative for the `Estimator` class is `StatevectorEstimator`.
  vqe_solver = VQE(Estimator(), ansatz,  P_BFGS(maxfun=1000, iprint=10) ) ## COBYLA(maxiter=2000, disp=True) # P_BFGS(maxfun=100, iprint=10) => 3m 21s, error 0.06611099896936112, NFT(maxiter=10, disp=True) => 50s,0.38112909977208664
For MacOS, python >= 3.8, using only current process. Multiple core use not supported.


RUNNING THE L-BFGS-B CODE

           * * *

Machine precision = 2.220D-16
 N =           26     M =           10

At X0         0 variables are exactly at the bounds

At iterate    0    f= -1.82914D+00    |proj g|=  2.81423D-01


 This problem is unconstrained.



At iterate   10    f= -1.99478D+00    |proj g|=  6.23945D-05

           * * *

Tit   = total number of iterations
Tnf   = total number of function evaluations
Tnint = total number of segments explored during Cauchy searches
Skip  = number of BFGS updates skipped
Nact  = number of active bounds at final generalized Cauchy point
Projg = norm of the final projected gradient
F     = final function value

           * * *

   N    Tit     Tnf  Tnint  Skip  Nact     Projg        F
   26     13     16      1     0     0   5.818D-06  -1.995D+00
  F =  -1.9947836197413278     

CONVERGENCE: NORM_OF_PROJECTED_GRADIENT_<=_PGTOL            
=== GROUND STATE ENERGY ===
 
* Electronic ground state energy (Hartree): -1.994783619741
  - computed part:      -1.994783619741
 
=== MEASURED OBSERVABLES ===
 
  0:  # Particles: 4.000 S: 0.000 S^2: 0.000 M: 0.000
 
=== DIPOLE MOMENTS ===
 
~ Nuclear dipole moment (a.u.): [17.00753512  0.0  0.0]
 
  0: 
  * Electronic dipole moment (a.u.): [17.007536149202

In [8]:
# res.hartree_fock_energy
ansatz.num_parameters, res.raw_result.cost_function_evals, res.raw_result.optimizer_time, res.eigenvalues[0], abs(true_lowest_ev - res.eigenvalues[0]), 

(26, 432, 43.22708487510681, -1.9947836197413278, 0.0013667057774817692)

In [14]:
for el,val in zip(ansatz.excitation_ops(), res.raw_result.optimal_point):
    print(el, ":",val)
    print()

Fermionic Operator
number spin orbitals=8, number terms=2
  1j * ( +_0 -_2 )
+ (-0-1j) * ( +_2 -_0 ) : 0.0055554200973386165

Fermionic Operator
number spin orbitals=8, number terms=2
  1j * ( +_0 -_3 )
+ (-0-1j) * ( +_3 -_0 ) : -3.0322679651069683e-07

Fermionic Operator
number spin orbitals=8, number terms=2
  1j * ( +_1 -_2 )
+ (-0-1j) * ( +_2 -_1 ) : 7.419157479054976e-07

Fermionic Operator
number spin orbitals=8, number terms=2
  1j * ( +_1 -_3 )
+ (-0-1j) * ( +_3 -_1 ) : 0.008046857311971387

Fermionic Operator
number spin orbitals=8, number terms=2
  1j * ( +_4 -_6 )
+ (-0-1j) * ( +_6 -_4 ) : 0.005317503271790873

Fermionic Operator
number spin orbitals=8, number terms=2
  1j * ( +_4 -_7 )
+ (-0-1j) * ( +_7 -_4 ) : -6.069860317854419e-08

Fermionic Operator
number spin orbitals=8, number terms=2
  1j * ( +_5 -_6 )
+ (-0-1j) * ( +_6 -_5 ) : -4.145680662389264e-07

Fermionic Operator
number spin orbitals=8, number terms=2
  1j * ( +_5 -_7 )
+ (-0-1j) * ( +_7 -_5 ) : 0.00810320700

In [18]:
len(ansatz.operators), ansatz.excitation_ops()

(26,
 [FermionicOp({'+_0 -_2': 1j, '+_2 -_0': (-0-1j)}, num_spin_orbitals=8, ),
  FermionicOp({'+_0 -_3': 1j, '+_3 -_0': (-0-1j)}, num_spin_orbitals=8, ),
  FermionicOp({'+_1 -_2': 1j, '+_2 -_1': (-0-1j)}, num_spin_orbitals=8, ),
  FermionicOp({'+_1 -_3': 1j, '+_3 -_1': (-0-1j)}, num_spin_orbitals=8, ),
  FermionicOp({'+_4 -_6': 1j, '+_6 -_4': (-0-1j)}, num_spin_orbitals=8, ),
  FermionicOp({'+_4 -_7': 1j, '+_7 -_4': (-0-1j)}, num_spin_orbitals=8, ),
  FermionicOp({'+_5 -_6': 1j, '+_6 -_5': (-0-1j)}, num_spin_orbitals=8, ),
  FermionicOp({'+_5 -_7': 1j, '+_7 -_5': (-0-1j)}, num_spin_orbitals=8, ),
  FermionicOp({'+_0 +_1 -_2 -_3': 1j, '+_3 +_2 -_1 -_0': (-0-1j)}, num_spin_orbitals=8, ),
  FermionicOp({'+_0 +_4 -_2 -_6': 1j, '+_6 +_2 -_4 -_0': (-0-1j)}, num_spin_orbitals=8, ),
  FermionicOp({'+_0 +_4 -_2 -_7': 1j, '+_7 +_2 -_4 -_0': (-0-1j)}, num_spin_orbitals=8, ),
  FermionicOp({'+_0 +_5 -_2 -_6': 1j, '+_6 +_2 -_5 -_0': (-0-1j)}, num_spin_orbitals=8, ),
  FermionicOp({'+_0 +_5 -_2 -_7

In [17]:
initial_state=HartreeFock(
        es_problem.num_spatial_orbitals,
        es_problem.num_particles,
        mapper,
    )
initial_state.draw()

┌───┐
q_0: ┤ X ├
     ├───┤
q_1: ┤ X ├
     └───┘
q_2: ─────
          
q_3: ─────
     ┌───┐
q_4: ┤ X ├
     ├───┤
q_5: ┤ X ├
     └───┘
q_6: ─────
          
q_7: ─────

In [ ]:
----- Result Summary -----
Method                  : VQE
Ansatz                  : UCCSD Minimal
No. of Pauli Observables: 197
Circuit Stats           : 3428 Gates with 26 parameters and 26 operators
Optimization terminated : Function tolerance reached
No. of function eval.   : 374
Evaluation Time         : 0 hrs 0 mins 0.8737 secs
Final objective value   : -1.9947774006793653
Final parameters:
  0 2^ :: 0.0054659728094905
  4 6^ :: 0.0049226449115728
  0 3^ :: 0.0000061638698454
  4 7^ :: -0.0000033116404708
  1 2^ :: -0.0000005425407249
  5 6^ :: -0.0000083216163372
  1 3^ :: 0.0075665785873179
  5 7^ :: 0.0080411623409297
  1 0 3^ 2^ :: 0.0560835169126948
  5 4 7^ 6^ :: 0.0560716117082524
  4 0 6^ 2^ :: -0.0949237121951570
  4 0 7^ 2^ :: -0.0000026299496902
  5 0 6^ 2^ :: -0.0000079584071585
  5 0 7^ 2^ :: 0.1897425855090117
  4 0 6^ 3^ :: 0.0000002839535081
  4 0 7^ 3^ :: -0.1627345293069382
  5 0 6^ 3^ :: 0.1200728750307267
  5 0 7^ 3^ :: 0.0000049878683227
  4 1 6^ 2^ :: 0.0000058331262501
  4 1 7^ 2^ :: 0.1200697326039308
  5 1 6^ 2^ :: -0.3611427210431494
  5 1 7^ 2^ :: 0.0000062086477152
  4 1 6^ 3^ :: 0.1936779727063531
  4 1 7^ 3^ :: 0.0000008674466299
  5 1 6^ 3^ :: -0.0000128046307483
  5 1 7^ 3^ :: -0.0807634047182180

In [10]:
n_sp = 4
count = 0
occupied = list(range(n_sp))
unoccupied = n_sp+np.array(list(range(n_sp)))
for p in unoccupied:
    for q in unoccupied:
        for r in occupied:
            for s in occupied:
                if p>q and q > r and r >s:
                    count += 1
                    print(count,"::" , p,q,r,s)

1 :: 5 4 1 0
2 :: 5 4 2 0
3 :: 5 4 2 1
4 :: 5 4 3 0
5 :: 5 4 3 1
6 :: 5 4 3 2
7 :: 6 4 1 0
8 :: 6 4 2 0
9 :: 6 4 2 1
10 :: 6 4 3 0
11 :: 6 4 3 1
12 :: 6 4 3 2
13 :: 6 5 1 0
14 :: 6 5 2 0
15 :: 6 5 2 1
16 :: 6 5 3 0
17 :: 6 5 3 1
18 :: 6 5 3 2
19 :: 7 4 1 0
20 :: 7 4 2 0
21 :: 7 4 2 1
22 :: 7 4 3 0
23 :: 7 4 3 1
24 :: 7 4 3 2
25 :: 7 5 1 0
26 :: 7 5 2 0
27 :: 7 5 2 1
28 :: 7 5 3 0
29 :: 7 5 3 1
30 :: 7 5 3 2
31 :: 7 6 1 0
32 :: 7 6 2 0
33 :: 7 6 2 1
34 :: 7 6 3 0
35 :: 7 6 3 1
36 :: 7 6 3 2


In [7]:
from scipy.stats import binomtest
binomtest(3, n=15, p=0.1).proportion_ci(confidence_level=0.95, method='wilson')

ConfidenceInterval(low=0.07047549346981558, high=0.4518544871516936)

In [5]:
ConfidenceInterval(low=0.07047549346981558, high=0.4518544871516936)

0.18406106910639106

In [ ]:
ConfidenceInterval(low=0.04331200510583602, high=0.48089113380685317)